
# 🤖 智能助手聊天机器人
这个笔记本提供了一个专业的AI智能助手，具有优雅的用户界面和完善的对话功能。

### 🌟 功能特性
1. **通用助手**: 能够回答各种领域的问题和提供多样化服务
2. **美观界面**: 现代化的用户界面设计，提升用户体验
3. **格式优化**: 完美支持markdown、代码等内容的格式化显示
4. **智能对话**: 支持上下文连续对话和历史记录管理
5. **数据导出**: 支持多种格式的对话历史导出

---

## 📋 使用步骤：
1. 在下方代码中配置您的OpenAI API密钥
2. 运行笔记本并开始与AI助手对话
3. 享受智能化的问答和咨询服务


In [1]:

from IPython.display import display, HTML

# 添加显示/隐藏代码的按钮和优化的样式
display(HTML('''
    <style>
        .input { display: none; }
        .toggle-button { 
            margin: 10px; 
            cursor: pointer; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 10px 20px;
            border: none;
            border-radius: 25px;
            font-weight: bold;
            box-shadow: 0 4px 15px rgba(0,0,0,0.2);
            transition: all 0.3s ease;
        }
        .toggle-button:hover {
            transform: translateY(-2px);
            box-shadow: 0 6px 20px rgba(0,0,0,0.3);
        }
        
        /* 优化整体界面样式 */
        .widget-area {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
        
        .jupyter-widgets {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
    </style>
    <div class="toggle-button" onclick="
        var code_cells = document.querySelectorAll('.input');
        code_cells.forEach(cell => {
            cell.style.display = cell.style.display === 'none' ? '' : 'none';
        });
    ">🔧 显示/隐藏代码</div>
'''))


In [2]:

# 安装必要的依赖库
import sys
import subprocess

def install(package):
    """安装Python包"""
    try:
        __import__(package)
    except ImportError:
        print(f"正在安装 {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])

# 依赖包列表
dependencies = ["ipywidgets", "openai", "markdown"]

for package in dependencies:
    install(package)

print("所有依赖包已安装完成。")


所有依赖包已安装完成。


In [ ]:

# 配置OpenAI API密钥（直接在代码中配置）
import os
from openai import OpenAI

# ================================
# 配置区域 - 请在此处设置您的API密钥
# ================================
OPENAI_API_KEY = "your-openai-api-key-here"  # 请替换为您的实际API密钥
MODEL_NAME = "gpt-4"  # 可选择: gpt-3.5-turbo, gpt-4, gpt-4-turbo 等
BASE_URL = "xxxx"

# 通用智能助手系统提示词
SYSTEM_PROMPT = """您是一个专业、友好、乐于助人的AI智能助手。您具有广泛的知识储备和强大的问题解决能力，能够为用户提供各种类型的帮助和咨询服务。

🎯 **核心能力：**
• 知识问答：回答各领域的专业问题和常识性问题
• 文本处理：写作、翻译、总结、分析等文本相关任务
• 问题解决：提供解决方案、建议和指导
• 学习辅导：协助学习、解释概念、提供教学支持
• 创意协作：头脑风暴、创意生成、方案设计

💡 **服务特色：**
• 准确可靠：基于事实提供准确信息
• 条理清晰：回答结构化、逻辑性强
• 友好耐心：以友善的态度对待每个问题
• 适应性强：根据用户需求调整回答风格
• 中文优先：优先使用中文进行交流

请以专业、友好、负责任的态度为用户提供高质量的智能助手服务。无论是学习、工作还是生活中的问题，我都会尽力为您提供有价值的帮助。"""
# ================================

# 验证API密钥是否已设置
if OPENAI_API_KEY == "your-openai-api-key-here" or not OPENAI_API_KEY:
    raise ValueError("请在上方配置区域设置您的OpenAI API密钥")

# 初始化OpenAI客户端
client = OpenAI(api_key=OPENAI_API_KEY, base_url=BASE_URL)

print("🚀 智能助手初始化成功")
print("💼 服务范围：通用问答、文本处理、学习辅导、创意协作")


🚀 智能助手初始化成功
💼 服务范围：通用问答、文本处理、学习辅导、创意协作


In [4]:

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Markdown
from datetime import datetime
import json
import re
import html

# 全局变量
conversation_history = []  # 存储对话历史
conversation_context = []  # 存储对话上下文，用于连续对话

# 创建优化的界面组件
username_input = widgets.Text(
    value="用户",
    description="👤 用户名:",
    layout=widgets.Layout(width="300px"),
    style={'description_width': '80px'}
)

user_input = widgets.Textarea(
    placeholder="💬 请输入您的问题...",
    layout=widgets.Layout(width="100%", height="120px"),
    style={'font_size': '14px'}
)

send_button = widgets.Button(
    description="📤 发送",
    button_style="success",
    icon="paper-plane",
    layout=widgets.Layout(width="120px", height="40px")
)

clear_button = widgets.Button(
    description="🗑️ 清空",
    button_style="warning",
    icon="refresh",
    layout=widgets.Layout(width="120px", height="40px")
)

save_button = widgets.Button(
    description="💾 保存",
    button_style="info",
    icon="download",
    layout=widgets.Layout(width="120px", height="40px")
)

# 创建美观的状态指示器
status_label = widgets.HTML(value="<span style='color: #4CAF50; font-weight: bold;'>✅ 就绪</span>")

# 创建优化的输出区域
output_area = widgets.Output(
    layout=widgets.Layout(
        border="2px solid #e0e0e0", 
        padding="20px", 
        width="100%", 
        height="500px",
        overflow="auto",
        border_radius="15px",
        background_color="#fafafa"
    )
)

# 对话上下文管理
context_toggle = widgets.Checkbox(
    value=True,
    description='🔗 保持对话上下文',
    disabled=False,
    indent=False,
    style={'description_width': '150px'}
)

max_context_length = widgets.IntSlider(
    value=10,
    min=1,
    max=20,
    step=1,
    description='📚 上下文长度:',
    layout=widgets.Layout(width="350px"),
    style={'description_width': '120px'}
)


def format_content(content):
    """优化内容格式化，正确处理markdown、代码等内容"""
    if not content:
        return content
    
    # 转义HTML特殊字符，但保留换行
    content = html.escape(content)
    
    # 处理代码块
    def replace_code_block(match):
        lang = match.group(1) or ''
        code = match.group(2)
        return f'''<div style="margin: 15px 0;"><div style="background: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; overflow: hidden;"><div style="background: #e9ecef; padding: 8px 15px; font-weight: bold; color: #495057; border-bottom: 1px solid #dee2e6;">💻 代码 ({lang or "文本"})</div><pre style="margin: 0; padding: 15px; background: #f8f9fa; overflow-x: auto; font-family: 'Consolas', 'Monaco', 'Courier New', monospace; font-size: 13px; line-height: 1.4;"><code>{code}</code></pre></div></div>'''
    
    # 处理行内代码
    def replace_inline_code(match):
        code = match.group(1)
        return f'<code style="background: #f1f3f4; padding: 2px 6px; border-radius: 4px; font-family: \'Consolas\', \'Monaco\', \'Courier New\', monospace; font-size: 13px; color: #d73a49;">{code}</code>'
    
    # 处理标题
    def replace_headers(match):
        level = len(match.group(1))
        text = match.group(2)
        sizes = {1: '24px', 2: '20px', 3: '18px', 4: '16px', 5: '14px', 6: '12px'}
        size = sizes.get(level, '16px')
        return f'<h{level} style="color: #2c3e50; margin: 20px 0 10px 0; font-size: {size}; font-weight: bold; border-bottom: 2px solid #3498db; padding-bottom: 5px;">{text}</h{level}>'
    
    # 处理列表
    def replace_list_items(match):
        marker = match.group(1)
        text = match.group(2)
        if marker in ['*', '-', '+']:
            return f'<li style="margin: 5px 0; line-height: 1.6;">• {text}</li>'
        else:  # 数字列表
            return f'<li style="margin: 5px 0; line-height: 1.6;">{text}</li>'
    
    # 处理粗体和斜体
    def replace_bold(match):
        text = match.group(1)
        return f'<strong style="color: #2c3e50; font-weight: bold;">{text}</strong>'
    
    def replace_italic(match):
        text = match.group(1)
        return f'<em style="font-style: italic; color: #34495e;">{text}</em>'
    
    # 应用格式化
    # 1. 代码块（三个反引号）
    content = re.sub(r'```(\w*)\n([\s\S]*?)```', replace_code_block, content)
    
    # 2. 行内代码
    content = re.sub(r'`([^`]+)`', replace_inline_code, content)
    
    # 3. 标题
    content = re.sub(r'^(#{1,6})\s+(.+)$', replace_headers, content, flags=re.MULTILINE)
    
    # 4. 粗体
    content = re.sub(r'\*\*([^*]+)\*\*', replace_bold, content)
    
    # 5. 斜体
    content = re.sub(r'\*([^*]+)\*', replace_italic, content)
    
    # 6. 列表项
    lines = content.split('\n')
    in_list = False
    result_lines = []
    
    for line in lines:
        # 检查是否是列表项
        list_match = re.match(r'^\s*([*+-]|\d+\.)\s+(.+)$', line)
        if list_match:
            if not in_list:
                result_lines.append('<ul style="margin: 10px 0; padding-left: 20px;">')
                in_list = True
            result_lines.append(replace_list_items(list_match))
        else:
            if in_list:
                result_lines.append('</ul>')
                in_list = False
            result_lines.append(line)
    
    if in_list:
        result_lines.append('</ul>')
    
    content = '\n'.join(result_lines)
    
    # 7. 处理换行
    content = content.replace('\n\n', '<br><br>').replace('\n', '<br>')
    
    return content


def chat_with_gpt(user_message, username):
    """与ChatGPT进行对话的核心函数"""
    try:
        # 构建消息列表
        messages = [{"role": "system", "content": SYSTEM_PROMPT}]
        
        # 如果启用上下文，添加历史对话
        if context_toggle.value and conversation_context:
            # 只保留最近的对话作为上下文
            recent_context = conversation_context[-max_context_length.value:]
            messages.extend(recent_context)
        
        # 添加当前用户消息
        messages.append({"role": "user", "content": user_message})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            max_tokens=2000,
            temperature=0.7,
            stream=False
        )
        
        assistant_response = response.choices[0].message.content
        
        # 更新对话上下文
        if context_toggle.value:
            conversation_context.append({"role": "user", "content": user_message})
            conversation_context.append({"role": "assistant", "content": assistant_response})
        
        return assistant_response
        
    except Exception as e:
        return f"❌ 与AI助手通信时发生错误: {str(e)}"


def format_message(username, message, is_user=True):
    """格式化消息显示，优化美观性"""
    timestamp = datetime.now().strftime("%H:%M:%S")
    formatted_content = format_content(message)
    
    if is_user:
        return f'''
        <div style="margin-bottom: 20px; padding: 0;">
            <div style="background: linear-gradient(135deg, #e3f2fd 0%, #bbdefb 100%); 
                        border-left: 5px solid #2196f3; 
                        border-radius: 15px; 
                        padding: 20px; 
                        box-shadow: 0 2px 10px rgba(0,0,0,0.1);
                        position: relative;
                        margin-left: 20px;">
                <div style="display: flex; align-items: center; margin-bottom: 10px;">
                    <span style="font-size: 20px; margin-right: 8px;">👤</span>
                    <strong style="color: #1976d2; font-size: 16px;">{username}</strong>
                    <span style="color: #666; font-size: 12px; margin-left: 10px;">({timestamp})</span>
                </div>
                <div style="color: #333; line-height: 1.6; font-size: 14px;">{formatted_content}</div>
            </div>
        </div>
        '''
    else:
        return f'''
        <div style="margin-bottom: 20px; padding: 0;">
            <div style="background: linear-gradient(135deg, #f3e5f5 0%, #e1bee7 100%); 
                        border-left: 5px solid #9c27b0; 
                        border-radius: 15px; 
                        padding: 20px; 
                        box-shadow: 0 2px 10px rgba(0,0,0,0.1);
                        position: relative;
                        margin-right: 20px;">
                <div style="display: flex; align-items: center; margin-bottom: 10px;">
                    <span style="font-size: 20px; margin-right: 8px;">🤖</span>
                    <strong style="color: #7b1fa2; font-size: 16px;">AI助手</strong>
                    <span style="color: #666; font-size: 12px; margin-left: 10px;">({timestamp})</span>
                </div>
                <div style="color: #333; line-height: 1.8; font-size: 14px;">{formatted_content}</div>
            </div>
        </div>
        '''


def handle_send_message(button):
    """处理发送消息事件"""
    user_message = user_input.value.strip()
    username = username_input.value.strip() or "用户"
    
    if not user_message:
        with output_area:
            display(HTML("<div style='color: #f44336; text-align: center; padding: 10px; background: #ffebee; border-radius: 8px; margin: 10px 0;'>⚠️ 请输入您的问题</div>"))
        return
    
    # 更新状态
    status_label.value = "<span style='color: #ff9800; font-weight: bold;'>🤔 AI助手正在思考...</span>"
    
    # 显示用户消息
    with output_area:
        display(HTML(format_message(username, user_message, True)))
    
    # 获取ChatGPT响应
    response = chat_with_gpt(user_message, username)
    
    # 显示ChatGPT响应
    with output_area:
        display(HTML(format_message("AI助手", response, False)))
    
    # 保存到历史记录
    conversation_history.append({
        "timestamp": datetime.now().isoformat(),
        "user": username,
        "user_message": user_message,
        "assistant_response": response
    })
    
    # 重置输入和状态
    user_input.value = ""
    status_label.value = "<span style='color: #4CAF50; font-weight: bold;'>✅ 就绪</span>"


def handle_clear_conversation(button):
    """清空对话历史"""
    global conversation_history, conversation_context
    conversation_history.clear()
    conversation_context.clear()
    output_area.clear_output()
    with output_area:
        display(HTML("<div style='color: #4CAF50; text-align: center; padding: 20px; background: #e8f5e8; border-radius: 10px; margin: 20px 0;'>✅ 对话已清空，可以开始新的对话</div>"))


def handle_save_history(button):
    """保存对话历史"""
    if not conversation_history:
        with output_area:
            display(HTML("<div style='color: #ff9800; text-align: center; padding: 10px; background: #fff3e0; border-radius: 8px; margin: 10px 0;'>⚠️ 没有对话历史可保存</div>"))
        return
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"对话历史_{timestamp}.md"
    
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write("# 🤖 AI助手对话历史\n\n")
            f.write(f"**生成时间:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            f.write(f"**总对话次数:** {len(conversation_history)}\n\n")
            f.write("---\n\n")
            
            for i, entry in enumerate(conversation_history, 1):
                f.write(f"## 📋 对话记录 {i}\n\n")
                f.write(f"**时间:** {entry['timestamp']}\n\n")
                f.write(f"**用户:** {entry['user']}\n\n")
                f.write(f"**问题:** {entry['user_message']}\n\n")
                f.write(f"**AI助手回复:**\n\n{entry['assistant_response']}\n\n")
                f.write("---\n\n")
        
        with output_area:
            display(HTML(f"<div style='color: #4CAF50; text-align: center; padding: 15px; background: #e8f5e8; border-radius: 10px; margin: 15px 0;'>✅ 对话历史已保存到: <strong>{filename}</strong></div>"))
            
    except Exception as e:
        with output_area:
            display(HTML(f"<div style='color: #f44336; text-align: center; padding: 10px; background: #ffebee; border-radius: 8px; margin: 10px 0;'>❌ 保存失败: {str(e)}</div>"))


# 绑定事件处理器
send_button.on_click(handle_send_message)
clear_button.on_click(handle_clear_conversation)
save_button.on_click(handle_save_history)

# 支持回车键发送（Ctrl+Enter）
def handle_key_press(change):
    # 这里可以添加键盘快捷键支持
    pass

user_input.observe(handle_key_press, names='value')

# 显示优化的界面
print("🚀 智能助手已启动")
print("💼 服务范围：通用问答、文本处理、学习辅导、创意协作")
print("✨ 界面优化：美观易用、格式完善、体验流畅")

# 用户信息和状态区域
display(widgets.HBox([
    username_input, 
    widgets.HTML(value="<div style='margin-left: 20px; display: flex; align-items: center;'></div>"),
    status_label
], layout=widgets.Layout(margin="10px 0")))

# 上下文设置区域
display(widgets.HBox([
    context_toggle, 
    max_context_length
], layout=widgets.Layout(margin="10px 0")))

# 输入区域
display(widgets.VBox([
    widgets.HTML(value="<h4 style='margin: 15px 0 5px 0; color: #2c3e50;'>💬 请输入您的问题：</h4>"),
    user_input
], layout=widgets.Layout(margin="10px 0")))

# 按钮区域
display(widgets.HBox([
    send_button, 
    widgets.HTML(value="<div style='width: 20px;'></div>"),
    clear_button, 
    widgets.HTML(value="<div style='width: 20px;'></div>"),
    save_button
], layout=widgets.Layout(margin="15px 0")))

# 对话显示区域
display(widgets.VBox([
    widgets.HTML(value="<h4 style='margin: 20px 0 10px 0; color: #2c3e50;'>💼 对话记录：</h4>"),
    output_area
]))

# 初始欢迎消息
with output_area:
    welcome_msg = """
    <div style='text-align: center; padding: 30px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 20px; margin-bottom: 20px; box-shadow: 0 8px 25px rgba(0,0,0,0.15);'>
        <h2 style='margin: 0 0 15px 0; font-size: 28px;'>🤖 欢迎使用智能助手</h2>
        <p style='margin: 10px 0; font-size: 16px; line-height: 1.6;'>我是您的AI智能助手，具备广泛的知识储备和强大的问题解决能力</p>
        <div style='background: rgba(255,255,255,0.2); border-radius: 15px; padding: 20px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; font-size: 18px;'>🎯 核心服务能力</h3>
            <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 10px; text-align: left;'>
                <div>💡 知识问答</div>
                <div>📝 文本处理</div>
                <div>🔧 问题解决</div>
                <div>📚 学习辅导</div>
                <div>🎨 创意协作</div>
                <div>🌐 多领域咨询</div>
            </div>
        </div>
        <p style='margin: 15px 0 0 0; font-size: 14px; opacity: 0.9;'>请在上方输入框中描述您的问题，我将为您提供专业的帮助和建议</p>
    </div>
    """
    display(HTML(welcome_msg))


🚀 智能助手已启动
💼 服务范围：通用问答、文本处理、学习辅导、创意协作
✨ 界面优化：美观易用、格式完善、体验流畅


In [5]:

# 额外功能：导出对话为JSON格式和统计分析
def export_conversation_json():
    """将对话历史导出为JSON格式"""
    if not conversation_history:
        with output_area:
            display(HTML("<div style='color: #ff9800; text-align: center; padding: 10px; background: #fff3e0; border-radius: 8px; margin: 10px 0;'>⚠️ 没有对话历史可导出</div>"))
        return
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"对话数据_{timestamp}.json"
    
    export_data = {
        "export_info": {
            "export_time": datetime.now().isoformat(),
            "total_conversations": len(conversation_history),
            "model_used": MODEL_NAME,
            "domain": "通用智能助手",
            "version": "2.0"
        },
        "conversations": conversation_history,
        "statistics": {
            "total_questions": len(conversation_history),
            "avg_response_length": sum(len(conv["assistant_response"]) for conv in conversation_history) / len(conversation_history) if conversation_history else 0,
            "users": list(set(conv["user"] for conv in conversation_history))
        }
    }
    
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(export_data, f, ensure_ascii=False, indent=2)
        
        with output_area:
            display(HTML(f"<div style='color: #4CAF50; text-align: center; padding: 15px; background: #e8f5e8; border-radius: 10px; margin: 15px 0;'>✅ 对话数据已导出为JSON格式: <strong>{filename}</strong><br/>📊 包含统计分析和完整对话记录</div>"))
    except Exception as e:
        with output_area:
            display(HTML(f"<div style='color: #f44336; text-align: center; padding: 10px; background: #ffebee; border-radius: 8px; margin: 10px 0;'>❌ 导出失败: {str(e)}</div>"))


def show_conversation_stats():
    """显示对话统计信息"""
    if not conversation_history:
        with output_area:
            display(HTML("<div style='color: #ff9800; text-align: center; padding: 10px; background: #fff3e0; border-radius: 8px; margin: 10px 0;'>⚠️ 暂无对话数据可分析</div>"))
        return
    
    total_convs = len(conversation_history)
    users = list(set(conv["user"] for conv in conversation_history))
    avg_response_length = sum(len(conv["assistant_response"]) for conv in conversation_history) / total_convs
    
    # 计算最活跃用户
    user_counts = {}
    for conv in conversation_history:
        user = conv["user"]
        user_counts[user] = user_counts.get(user, 0) + 1
    
    most_active_user = max(user_counts, key=user_counts.get) if user_counts else "无"
    
    stats_html = f"""
    <div style='background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); color: white; padding: 25px; border-radius: 15px; margin: 20px 0; box-shadow: 0 6px 20px rgba(0,0,0,0.15);'>
        <h3 style='margin: 0 0 20px 0; text-align: center;'>📊 对话统计分析</h3>
        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px;'>
            <div style='background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px;'>
                <h4 style='margin: 0 0 10px 0;'>💬 对话总量</h4>
                <p style='font-size: 24px; font-weight: bold; margin: 0;'>{total_convs} 次</p>
            </div>
            <div style='background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px;'>
                <h4 style='margin: 0 0 10px 0;'>👥 用户数量</h4>
                <p style='font-size: 24px; font-weight: bold; margin: 0;'>{len(users)} 人</p>
            </div>
            <div style='background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px;'>
                <h4 style='margin: 0 0 10px 0;'>📝 平均回复长度</h4>
                <p style='font-size: 20px; font-weight: bold; margin: 0;'>{avg_response_length:.0f} 字符</p>
            </div>
            <div style='background: rgba(255,255,255,0.2); padding: 15px; border-radius: 10px;'>
                <h4 style='margin: 0 0 10px 0;'>🏆 最活跃用户</h4>
                <p style='font-size: 18px; font-weight: bold; margin: 0;'>{most_active_user}</p>
                <p style='font-size: 14px; margin: 5px 0 0 0;'>({user_counts.get(most_active_user, 0)} 次对话)</p>
            </div>
        </div>
    </div>
    """
    
    with output_area:
        display(HTML(stats_html))


# 创建额外功能按钮
export_json_button = widgets.Button(
    description="📊 导出JSON",
    button_style="primary",
    icon="file-code",
    layout=widgets.Layout(width="140px", height="40px")
)

stats_button = widgets.Button(
    description="📈 查看统计",
    button_style="info",
    icon="bar-chart",
    layout=widgets.Layout(width="140px", height="40px")
)

def handle_export_json(button):
    export_conversation_json()

def handle_show_stats(button):
    show_conversation_stats()

export_json_button.on_click(handle_export_json)
stats_button.on_click(handle_show_stats)

print("\n📁 高级功能：")
display(widgets.HBox([
    export_json_button,
    widgets.HTML(value="<div style='width: 20px;'></div>"),
    stats_button
], layout=widgets.Layout(margin="10px 0")))

print("💾 数据管理：支持JSON格式导出，便于数据分析和系统集成")
print("📊 统计分析：实时查看对话量、用户活跃度等关键指标")



📁 高级功能：


💾 数据管理：支持JSON格式导出，便于数据分析和系统集成
📊 统计分析：实时查看对话量、用户活跃度等关键指标
